<a href="https://colab.research.google.com/github/meatflavourdev/tortoise-tts/blob/main/tortoise_tts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Welcome to Tortoise! 🐢🐢🐢🐢

Before you begin, I **strongly** recommend you turn on a GPU runtime.

There's a reason this is called "Tortoise" - this model takes up to a minute to perform inference for a single sentence on a GPU. Expect waits on the order of hours on a CPU.

In [2]:
!git clone https://github.com/neonbjb/tortoise-tts.git
%cd tortoise-tts
!pip3 install -r requirements.txt
!python3 setup.py install

fatal: destination path 'tortoise-tts' already exists and is not an empty directory.
/content/tortoise-tts
running install
running bdist_egg
running egg_info
writing TorToiSe.egg-info/PKG-INFO
writing dependency_links to TorToiSe.egg-info/dependency_links.txt
writing requirements to TorToiSe.egg-info/requires.txt
writing top-level names to TorToiSe.egg-info/top_level.txt
adding license file 'LICENSE'
writing manifest file 'TorToiSe.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/tortoise
copying build/lib/tortoise/api.py -> build/bdist.linux-x86_64/egg/tortoise
copying build/lib/tortoise/__init__.py -> build/bdist.linux-x86_64/egg/tortoise
copying build/lib/tortoise/do_tts.py -> build/bdist.linux-x86_64/egg/tortoise
copying build/lib/tortoise/read.py -> build/bdist.linux-x86_64/egg/tortoise
copying build/lib/tortoise/is_this_from_tortoise.py -> 

In [3]:
# Imports used through the rest of the notebook.
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F

import IPython

from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices

# This will download all the models used by Tortoise from the HF hub.
tts = TextToSpeech()

In [17]:
# This is the text that will be spoken.
text = "Joining two modalities results in a surprising increase in generalization! What would happen if we combined them all?"

# Here's something for the poetically inclined.. (set text=)
"""
Then took the other, as just as fair,
And having perhaps the better claim,
Because it was grassy and wanted wear;
Though as for that the passing there
Had worn them really about the same,"""

# Pick a "preset mode" to determine quality. Options: {"ultra_fast", "fast" (default), "standard", "high_quality"}. See docs in api.py
preset = "fast"

In [18]:
# Tortoise will attempt to mimic voices you provide. It comes pre-packaged
# with some voices you might recognize.

# Let's list all the voices available. These are just some random clips I've gathered
# from the internet as well as a few voices from the training dataset.
# Feel free to add your own clips to the voices/ folder.
%ls tortoise/voices

IPython.display.Audio('tortoise/voices/tom/1.wav')

angie/      freeman/  mol/      snakes/        train_dotrice/  train_lescault/
applejack/  geralt/   myself/   tim_reynolds/  train_dreams/   train_mouse/
daniel/     halle/    pat/      tom/           train_empire/   weaver/
deniro/     jlaw/     pat2/     train_atkins/  train_grace/    william/
emma/       lj/       rainbow/  train_daws/    train_kennard/


In [20]:
# Pick one of the voices from the output above
voice = 'jeremy'

# Load it and send it through Tortoise.
voice_samples, conditioning_latents = load_voice(voice)
gen = tts.tts_with_preset("I am trying to pronounce EnEffTees without sounding like an idiot. Is this how you say EnEffTee?", voice_samples=voice_samples, conditioning_latents=conditioning_latents, 
                          preset=preset)
torchaudio.save('generated.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio('generated.wav')

Generating autoregressive samples..


100%|██████████| 6/6 [01:12<00:00, 12.14s/it]


Computing best candidates using CLVP and CVVP


  0%|          | 0/6 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
100%|██████████| 6/6 [00:16<00:00,  2.72s/it]


Transforming autoregressive outputs into audio..


100%|██████████| 80/80 [00:38<00:00,  2.07it/s]


In [ ]:
# Tortoise can also generate speech using a random voice. The voice changes each time you execute this!
# (Note: random voices can be prone to strange utterances)
gen = tts.tts_with_preset(text, voice_samples=None, conditioning_latents=None, preset=preset)
torchaudio.save('generated.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio('generated.wav')

Generating autoregressive samples..


100%|██████████| 6/6 [01:22<00:00, 13.71s/it]


Computing best candidates using CLVP and CVVP


  0%|          | 0/6 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
100%|██████████| 6/6 [00:03<00:00,  1.75it/s]


Transforming autoregressive outputs into audio..


100%|██████████| 80/80 [00:39<00:00,  2.03it/s]


In [ ]:
# You can also combine conditioning voices. Combining voices produces a new voice
# with traits from all the parents.
#
# Lets see what it would sound like if Picard and Kirk had a kid with a penchant for philosophy:
voice_samples, conditioning_latents = load_voices(['pat', 'william'])

gen = tts.tts_with_preset("They used to say that if man was meant to fly, he’d have wings. But he did fly. He discovered he had to.", 
                          voice_samples=None, conditioning_latents=None, preset=preset)
torchaudio.save('captain_kirkard.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio('captain_kirkard.wav')

Generating autoregressive samples..


100%|██████████| 6/6 [01:03<00:00, 10.58s/it]


Computing best candidates using CLVP and CVVP


  0%|          | 0/6 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
100%|██████████| 6/6 [00:03<00:00,  1.80it/s]


Transforming autoregressive outputs into audio..


100%|██████████| 80/80 [00:34<00:00,  2.35it/s]


In [ ]:
# This will download all the models used by Tortoise from the HF hub.
tts = TextToSpeech()

In [4]:
del tts  # Will break other cells, but necessary to conserve RAM if you want to run this cell.

# Tortoise comes with some scripts that does a lot of the lifting for you. For example,
# read.py will read a text file for you.
!python3 tortoise/read.py --voice=jeremy --textfile=tortoise/data/nfts-01.txt --preset=ultra_fast --output_path=.

IPython.display.Audio('jeremy/combined.wav')
# This will take awhile..

Generating autoregressive samples..
100% 2/2 [01:29<00:00, 44.69s/it]
Computing best candidates using CLVP and CVVP
  0% 0/2 [00:00<?, ?it/s]No stop tokens found in one of the generated voice clips. This typically means the spoken audio is too long. In some cases, the output will still be good, though. Listen to it and if it is missing words, try breaking up your input text.
No stop tokens found in one of the generated voice clips. This typically means the spoken audio is too long. In some cases, the output will still be good, though. Listen to it and if it is missing words, try breaking up your input text.
No stop tokens found in one of the generated voice clips. This typically means the spoken audio is too long. In some cases, the output will still be good, though. Listen to it and if it is missing words, try breaking up your input text.
No stop tokens found in one of the generated voice clips. This typically means the spoken audio is too long. In some cases, the output will still be

In [6]:

IPython.display.Audio('jeremy/combined.wav')